In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps 
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing 
%matplotlib inline

In [2]:
%%time
data=pd.read_csv('data60.csv')
oot=pd.read_csv('oot.csv')

CPU times: user 6.71 s, sys: 1.02 s, total: 7.72 s
Wall time: 7.92 s


In [3]:
can_var=['fraud_label','Days_since_fulladdress', 'fulladdress30', 'fulladdress14', 
         'fulladdress7', 'fulladdress3', 'fulladdress1', 'Days_since_ssn', 
         'ssn30', 'Days_since_firstname_ssn', 'Days_since_lastname_ssn', 
         'Days_since_fulladdresshomephone', 'Days_since_nameDOB', 'ssnnameDOB30', 
         'Days_since_ssnnameDOB', 'fulladdresshomephone30', 'nameDOB30',
         'lastnamessn30', 'firstnamessn30', 'fulladdresshomephone14',
         'nameDOB14', 'ssnnameDOB14', 'fulladdresshomephone7', 'nameDOB7', 
         'ssn7', 'homephone3', 'homephone7','zip3_risk']

In [4]:
data1=data.loc[:, can_var]
oot1=oot.loc[:,can_var]

In [5]:
data1.head()

,fraud_label,Days_since_fulladdress,fulladdress30,fulladdress14,fulladdress7,fulladdress3,fulladdress1,Days_since_ssn,ssn30,Days_since_firstname_ssn,...,firstnamessn30,fulladdresshomephone14,nameDOB14,ssnnameDOB14,fulladdresshomephone7,nameDOB7,ssn7,homephone3,homephone7,zip3_risk
0,0,365,1,1,1,1,1,365,1,365,...,1,1,1,1,1,1,1,1,1,0.01252
1,0,365,1,1,1,1,1,365,1,365,...,1,1,1,1,1,1,1,1,1,0.01252
2,0,365,1,1,1,1,1,365,1,365,...,1,1,1,1,1,1,1,1,2,0.01252
3,0,365,1,1,1,1,1,365,1,365,...,1,1,1,1,1,1,1,1,1,0.01252
4,0,365,1,1,1,1,1,365,1,365,...,1,1,1,1,1,1,1,1,1,0.01252


In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    data1,
    test_size=0.3,
    random_state=1)

In [7]:
import math
import matplotlib as mpl
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from random import sample


In [8]:
# data2:all goods, data3:all bads
data2 = train[train['fraud_label']==0]
data3 = train[train['fraud_label']==1]

In [25]:
train['fraud_label'].sum()

8377

In [26]:
len(train)

583454

In [27]:
8377/(583454-8377)

0.014566744975020736

In [11]:
#for balanced dataset
data4,data5 = train_test_split(
    data2,
    test_size=0.9,
    random_state=1)
data5 = data3.append(data4) 

In [14]:
data5['fraud_label'].sum()

8377

In [18]:
len(data5)

65884

In [17]:
8377/65884

0.12714771416428874

In [23]:
(65884-8377)/65884

0.8728522858357113

In [24]:
8377/(65884-8377)

0.14566922287721495

In [ ]:
X1_train = train.iloc[:,1:]
y1_train = train.iloc[:,0]
X1_test = test.iloc[:,1:]
y1_test = test.iloc[:,0]
X1_oot = oot1.iloc[:,1:]
y1_oot = oot1.iloc[:,0]

In [ ]:
#model
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2, 3), random_state=1)
nn.fit(X1_train, y1_train)

In [ ]:
#test
nn1_prob_test=nn.predict_proba(X1_test)
nn_test_df = pd.DataFrame(nn1_prob_test,index=y1_test.index,columns = ['prob', 'y'])
nn_test_df['y']=y1_test

In [ ]:
k=0
cur=0
num=int(0.03*len(nn_test_df.index))
for i in nn_test_df.sort_values(by='prob').index:
    if cur<num:
        cur+=1
        k+=nn_test_df.loc[i,'y']
# test
k/sum(nn_test_df['y'])

In [ ]:
#train
nn1_prob_train=nn.predict_proba(X1_train)
nn_train_df = pd.DataFrame(nn1_prob_train,index=y1_train.index,columns = ['prob', 'y'])
nn_train_df['y']=y1_train

In [ ]:
k=0
cur=0
num=int(0.03*len(nn_train_df.index))
for i in nn_train_df.sort_values(by='prob').index:
    if cur<num:
        cur+=1
        k+=nn_train_df.loc[i,'y']
# train
10*k/sum(nn_train_df['y'])

In [ ]:
#oot
nn1_prob_oot=nn.predict_proba(X1_oot)
nn_oot_df = pd.DataFrame(nn1_prob_oot,index=y1_oot.index,columns = ['prob', 'y'])
nn_oot_df['y']=y1_oot

In [ ]:
k=0
cur=0
num=int(0.03*len(nn_oot_df.index))
for i in nn_oot_df.sort_values(by='prob').index:
    if cur<num:
        cur+=1
        k+=nn_oot_df.loc[i,'y']
# oot FDR
k/sum(nn_oot_df['y'])

In [69]:
FDR_train_list={}
FDR_test_list={}
FDR_oot_list={}
for s in range(0,10):
    train, test = train_test_split(
    data1,
    test_size=0.3,
    random_state=s)
    X1_train = train.iloc[:,1:]
    y1_train = train.iloc[:,0]
    X1_test = test.iloc[:,1:]
    y1_test = test.iloc[:,0]
    X1_oot = oot1.iloc[:,1:]
    y1_oot = oot1.iloc[:,0]
    from sklearn.neural_network import MLPClassifier
    nn = MLPClassifier(activation='logistic',  batch_size='auto',
            early_stopping=True, hidden_layer_sizes=(2,3), learning_rate='adaptive',
            learning_rate_init=0.3, max_iter=1000, random_state=1,
            solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
            warm_start=False)
    nn.fit(X1_train, y1_train)
    #train
    nn1_prob_train=nn.predict_proba(X1_train)
    nn_train_df = pd.DataFrame(nn1_prob_train,index=y1_train.index,columns = ['prob', 'y'])
    nn_train_df['y']=y1_train
    k_train=0
    cur_train=0
    num_train=int(0.03*len(nn_train_df.index))
    for i in nn_train_df.sort_values(by='prob').index:
        if cur_train<num_train:
            cur_train+=1
            k_train+=nn_train_df.loc[i,'y']
    # FDR
    FDR_train=k_train/sum(nn_train_df['y'])
    #test
    nn1_prob_test=nn.predict_proba(X1_test)
    nn_test_df = pd.DataFrame(nn1_prob_test,index=y1_test.index,columns = ['prob', 'y'])
    nn_test_df['y']=y1_test
    #FDR
    k_test=0
    cur_test=0
    num_test=int(0.03*len(nn_test_df.index))
    for i in nn_test_df.sort_values(by='prob').index:
        if cur_test<num_test:
            cur_test+=1
            k_test+=nn_test_df.loc[i,'y']
    # test FDR
    FDR_test=k_test/sum(nn_test_df['y'])
    #OOT
    nn1_prob_oot=nn.predict_proba(X1_oot)
    nn_oot_df = pd.DataFrame(nn1_prob_oot,index=y1_oot.index,columns = ['prob', 'y'])
    nn_oot_df['y']=y1_oot
    #OOT FDR
    k_oot=0
    cur_oot=0
    num_oot=int(0.03*len(nn_oot_df.index))
    for i in nn_oot_df.sort_values(by='prob').index:
        if cur_oot<num_oot:
            cur_oot+=1
            k_oot+=nn_oot_df.loc[i,'y']
    # oot FDR
    FDR_oot=k_oot/sum(nn_oot_df['y'])
    FDR_train_list[s]=FDR_train
    FDR_test_list[s]=FDR_test
    FDR_oot_list[s]=FDR_oot

In [ ]:
MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2,3), random_state=1,activation='logistic',
early_stopping=True)

In [ ]:
MLPClassifier(activation='logistic',  batch_size='auto',
            early_stopping=True, hidden_layer_sizes=(2,3), learning_rate='adaptive',
            learning_rate_init=0.1, max_iter=5000, random_state=1,
            solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
            warm_start=False)

In [ ]:
{0: 0.5718040621266428, 1: 0.5697826345171636, 2: 0.5434052757793765, 3: 0.5451940298507463, 4: 0.5375639957137754, 5: 0.5399621659966896, 6: 0.5557382826265711, 7: 0.5745489078822412, 8: 0.5371617629682146, 9: 0.5331264175719231}
{0: 0.5680505911465493, 1: 0.568561872909699, 2: 0.5235887646577584, 3: 0.5412995594713657, 4: 0.5385254988913526, 5: 0.535643843336151, 6: 0.5592444190040069, 7: 0.574937203460787, 8: 0.5428893905191874, 9: 0.5564738292011019}
{0: 0.5511316010058676, 1: 0.5452640402347024, 2: 0.5238893545683152, 3: 0.5284995808885163, 4: 0.5251466890192791, 5: 0.5255658005029338, 6: 0.5230511316010059, 7: 0.5586756077116513, 8: 0.5247275775356245, 9: 0.5243084660519698}

In [70]:
print(FDR_train_list)
print(FDR_test_list)
print(FDR_oot_list)

{0: 0.515531660692951, 1: 0.526071980045136, 2: 0.5213429256594724, 3: 0.5204776119402985, 4: 0.5216097154423146, 5: 0.5115866635138331, 6: 0.5322448020674263, 7: 0.51661918328585, 8: 0.5130568356374808, 9: 0.510684015757431}
{0: 0.5174594445971955, 1: 0.5256410256410257, 2: 0.5012271611671666, 3: 0.5134911894273128, 4: 0.5194013303769401, 5: 0.5085939701324317, 6: 0.5343445907269605, 7: 0.5166061959252023, 8: 0.5191873589164786, 9: 0.5377410468319559}
{0: 0.4865884325230511, 1: 0.5067057837384744, 2: 0.4911986588432523, 3: 0.4911986588432523, 4: 0.4886839899413244, 5: 0.49077954735959767, 6: 0.5104777870913663, 7: 0.4853310980720872, 8: 0.49245599329421624, 9: 0.4979044425817267}


In [71]:
#balanced train dataset
FDR_train_balancedlist={}
FDR_test_balancedlist={}
FDR_oot_balancedlist={}
for s in range(0,10):
    train, test = train_test_split(
    data1,
    test_size=0.3,
    random_state=s)
    data2 = train[train['fraud_label']==0]
    data3 = train[train['fraud_label']==1]
    data4,data5 = train_test_split(
    data2,
    test_size=0.9,
    random_state=s)
    data5 = data3.append(data4) 
    X1_train = data5.iloc[:,1:]
    y1_train = data5.iloc[:,0]
    X1_test = test.iloc[:,1:]
    y1_test = test.iloc[:,0]
    X1_oot = oot1.iloc[:,1:]
    y1_oot = oot1.iloc[:,0]
    from sklearn.neural_network import MLPClassifier
    nn = MLPClassifier(activation='logistic',  batch_size='auto',
            early_stopping=True, hidden_layer_sizes=(2,3), learning_rate='adaptive',
            learning_rate_init=0.3, max_iter=1000, random_state=1,
            solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
            warm_start=False)
    nn.fit(X1_train, y1_train)
    #train
    nn1_prob_train=nn.predict_proba(X1_train)
    nn_train_df = pd.DataFrame(nn1_prob_train,index=y1_train.index,columns = ['prob', 'y'])
    nn_train_df['y']=y1_train
    k_train=0
    cur_train=0
    num_train=int(0.03*len(train))
    for i in nn_train_df.sort_values(by='prob').index:
        if cur_train<num_train:
            cur_train+=1
            k_train+=nn_train_df.loc[i,'y']
    # FDR
    FDR_train=k_train/sum(nn_train_df['y'])
    #test
    nn1_prob_test=nn.predict_proba(X1_test)
    nn_test_df = pd.DataFrame(nn1_prob_test,index=y1_test.index,columns = ['prob', 'y'])
    nn_test_df['y']=y1_test
    #FDR
    k_test=0
    cur_test=0
    num_test=int(0.03*len(nn_test_df.index))
    for i in nn_test_df.sort_values(by='prob').index:
        if cur_test<num_test:
            cur_test+=1
            k_test+=nn_test_df.loc[i,'y']
    # test FDR
    FDR_test=k_test/sum(nn_test_df['y'])
    #OOT
    nn1_prob_oot=nn.predict_proba(X1_oot)
    nn_oot_df = pd.DataFrame(nn1_prob_oot,index=y1_oot.index,columns = ['prob', 'y'])
    nn_oot_df['y']=y1_oot
    #OOT FDR
    k_oot=0
    cur_oot=0
    num_oot=int(0.03*len(nn_oot_df.index))
    for i in nn_oot_df.sort_values(by='prob').index:
        if cur_oot<num_oot:
            cur_oot+=1
            k_oot+=nn_oot_df.loc[i,'y']
    # oot FDR
    FDR_oot=k_oot/sum(nn_oot_df['y'])
    FDR_train_balancedlist[s]=FDR_train
    FDR_test_balancedlist[s]=FDR_test
    FDR_oot_balancedlist[s]=FDR_oot

In [72]:
print(FDR_train_balancedlist)
print(FDR_test_balancedlist)
print(FDR_oot_balancedlist)

{0: 0.5326164874551972, 1: 0.5219147167121986, 2: 0.5406474820143885, 3: 0.5364776119402985, 4: 0.6550779854744613, 5: 0.5244738708914637, 6: 0.5139198872312933, 7: 0.5274216524216524, 8: 0.5255819449367837, 9: 0.6516652739644264}
{0: 0.5367060764366236, 1: 0.5278706800445931, 2: 0.5222252522497954, 3: 0.5324889867841409, 4: 0.5512749445676275, 5: 0.5291631445477599, 6: 0.5197481396680023, 7: 0.5333519397153224, 8: 0.5372460496613995, 9: 0.5611570247933885}
{0: 0.5209555741827326, 1: 0.4945515507124895, 2: 0.5171835708298408, 3: 0.5217937971500419, 4: 0.5222129086336965, 5: 0.5163453478625314, 6: 0.49706621961441744, 7: 0.5138306789606035, 8: 0.510896898575021, 9: 0.5255658005029338}


In [ ]:
#run for loop to test parameter

In [6]:
result_oot={}
train, test = train_test_split(
data1,
test_size=0.3,
random_state=1)
data2 = train[train['fraud_label']==0]
data3 = train[train['fraud_label']==1]
data4,data5 = train_test_split(
data2,
test_size=0.9,
random_state=1)
data5 = data3.append(data4) 
X1_train = data5.iloc[:,1:]
y1_train = data5.iloc[:,0]
X1_test = test.iloc[:,1:]
y1_test = test.iloc[:,0]
X1_oot = oot1.iloc[:,1:]
y1_oot = oot1.iloc[:,0]

In [27]:
for j in range(1,26):
    for k in range(1,26):
        from sklearn.neural_network import MLPClassifier
        nn = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(j,k), random_state=1)
        nn.fit(X1_train, y1_train)
        #OOT
        nn1_prob_oot=nn.predict_proba(X1_oot)
        nn_oot_df = pd.DataFrame(nn1_prob_oot,index=y1_oot.index,columns = ['prob', 'y'])
        nn_oot_df['y']=y1_oot
        #OOT FDR
        k_oot=0
        cur_oot=0
        num_oot=int(0.03*len(nn_oot_df.index))
        for i in nn_oot_df.sort_values(by='prob').index:
            if cur_oot<num_oot:
                cur_oot+=1
                k_oot+=nn_oot_df.loc[i,'y']
        # oot FDR
        fdr=k_oot/sum(nn_oot_df['y'])
        print(j,k,fdr)


1 1 0.02388935456831517
1 2 0.02388935456831517
1 3 0.18943839061190276
1 4 0.02388935456831517
1 5 0.11106454316848281
1 6 0.3616932103939648
1 7 0.02388935456831517
1 8 0.02388935456831517
1 9 0.02388935456831517
1 10 0.02388935456831517
1 11 0.02388935456831517
1 12 0.02388935456831517
1 13 0.20368818105616093
1 14 0.2355406538139145
1 15 0.02388935456831517
1 16 0.02388935456831517
1 17 0.10980720871751885
1 18 0.02388935456831517
1 19 0.02388935456831517
1 20 0.02388935456831517
1 21 0.02388935456831517
1 22 0.02388935456831517
1 23 0.02388935456831517
1 24 0.02388935456831517
1 25 0.02388935456831517
2 1 0.30511316010058676
2 2 0.4878457669740151
2 3 0.5167644593461861
2 4 0.4782062028499581
2 5 0.3721709974853311
2 6 0.02388935456831517
2 7 0.4865884325230511
2 8 0.5134115674769488
2 9 0.45892707460184406
2 10 0.5117351215423303
2 11 0.3688181056160939
2 12 0.48113998323554064
2 13 0.5025146689019279
2 14 0.02388935456831517
2 15 0.35875943000838223
2 16 0.3608549874266555
2 17 

14 13 0.4953897736797988
14 14 0.21207041072925398
14 15 0.4782062028499581
14 16 0.48197820620285
14 17 0.48113998323554064
14 18 0.510896898575021
14 19 0.2933780385582565
14 20 0.5050293378038558
14 21 0.28331936295054483
14 22 0.49706621961441744
14 23 0.2531433361274099
14 24 0.5
14 25 0.508382229673093
15 1 0.02388935456831517
15 2 0.510896898575021
15 3 0.02388935456831517
15 4 0.4974853310980721
15 5 0.48575020955574183
15 6 0.49874266554903607
15 7 0.5163453478625314
15 8 0.42875104777870915
15 9 0.49161777032690696
15 10 0.4974853310980721
15 11 0.450544844928751
15 12 0.45473595976529757
15 13 0.32984073763621125
15 14 0.4316848281642917
15 15 0.432523051131601
15 16 0.433780385582565
15 17 0.43126571668063707
15 18 0.5176026823134954
15 19 0.4404861693210394
15 20 0.4832355406538139
15 21 0.45892707460184406
15 22 0.5075440067057837
15 23 0.06496227996647108
15 24 0.4920368818105616
15 25 0.055322715842414084
16 1 0.02388935456831517
16 2 0.5004191114836547
16 3 0.023470243

In [59]:
from sklearn.neural_network import MLPClassifier
for k in range(1,26):
    nn = MLPClassifier(activation='logistic',  batch_size='auto',
            early_stopping=True, hidden_layer_sizes=(k,), learning_rate='adaptive',
            learning_rate_init=0.3, max_iter=1000, random_state=1,
            solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
            warm_start=False)
    nn.fit(X1_train, y1_train)
    #OOT
    nn1_prob_oot=nn.predict_proba(X1_oot)
    nn_oot_df = pd.DataFrame(nn1_prob_oot,index=y1_oot.index,columns = ['prob', 'y'])
    nn_oot_df['y']=y1_oot
    #OOT FDR
    k_oot=0
    cur_oot=0
    num_oot=int(0.03*len(nn_oot_df.index))
    for i in nn_oot_df.sort_values(by='prob').index:
        if cur_oot<num_oot:
            cur_oot+=1
            k_oot+=nn_oot_df.loc[i,'y']
    # oot FDR
    fdr=k_oot/sum(nn_oot_df['y'])
    print(k,fdr)

1 0.29044425817267394
2 0.5037720033528919
3 0.3583403185247276
4 0.5071248952221291
5 0.3683989941324392
6 0.5150880134115675
7 0.5180217937971501
8 0.5167644593461861
9 0.5431684828164292
10 0.4932942162615256
11 0.5230511316010059
12 0.5159262363788768
13 0.5381391450125733
14 0.528918692372171
15 0.5565800502933781
16 0.5565800502933781
17 0.528918692372171
18 0.5561609388097234
19 0.5553227158424141
20 0.5515507124895223
21 0.5553227158424141
22 0.548197820620285
23 0.5477787091366303
24 0.5523889354568315
25 0.5490360435875943


In [28]:
for k in range(1,26):
    from sklearn.neural_network import MLPClassifier
    nn = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(39,k), random_state=1)
    nn.fit(X1_train, y1_train)
    #OOT
    nn1_prob_oot=nn.predict_proba(X1_oot)
    nn_oot_df = pd.DataFrame(nn1_prob_oot,index=y1_oot.index,columns = ['prob', 'y'])
    nn_oot_df['y']=y1_oot
    #OOT FDR
    k_oot=0
    cur_oot=0
    num_oot=int(0.03*len(nn_oot_df.index))
    for i in nn_oot_df.sort_values(by='prob').index:
        if cur_oot<num_oot:
            cur_oot+=1
            k_oot+=nn_oot_df.loc[i,'y']
    # oot FDR
    fdr=k_oot/sum(nn_oot_df['y'])
    print(39,k,fdr)

39 1 0.02388935456831517
39 2 0.02388935456831517
39 3 0.02388935456831517
39 4 0.32606873428331934
39 5 0.4543168482816429
39 6 0.4878457669740151
39 7 0.4371332774518022
39 8 0.20955574182732606
39 9 0.46186085498742663
39 10 0.4559932942162615
39 11 0.24056999161777032
39 12 0.3348700754400671
39 13 0.4861693210393965
39 14 0.4974853310980721
39 15 0.49832355406538137
39 16 0.49497066219614416
39 17 0.49832355406538137
39 18 0.2585917854149204
39 19 0.4966471081307628
39 20 0.5004191114836547
39 21 0.5008382229673093
39 22 0.5037720033528919
39 23 0.49077954735959767
39 24 0.5020955574182733
39 25 0.24685666387259012
